## Erstellung eines neuronalen Netzes zur Klassifikation von Musik
Wie trainiert man ein neuronales Netzwerk, um das Genre eines Songs zu erkennen? In dieser Aufgabe wird  
Schritt für schritt ein neuronales Netzwerk mithilfe von bereitgestellten extrahierten Features trainiert.  
Nach dem Training wird die Genauigkeit des Modells überprüft, um festzustellen, wie gut es in der Lage ist,  
bestimmte Genres zu erkennen.

</div>

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 1:</b>
    
Importieren Sie alle benötigten Bibliotheken.
    
</div>

In [43]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 2:</b>
    
Laden Sie die Datei data.csv mit Hilfe der Pandas-Bibliothek und der Funktion read_csv() in einen Dataframe.
Zeigen Sie die ersten Zeilen der CSV an.
    
</div>

In [44]:
# code
import pandas as pd

# Lesen der CSV-Datei in ein DataFrame
df = pd.read_csv('data.csv')

# Ausgabe der ersten Zeilen des DataFrames
print(df.head())


          filename  length  chroma_stft_mean  chroma_stft_var  rms_mean  \
0  blues.00000.wav      30          0.349951         0.088752  0.130192   
1  blues.00001.wav      30          0.340945         0.094968  0.095892   
2  blues.00002.wav      30          0.363562         0.085277  0.175494   
3  blues.00003.wav      30          0.404848         0.094018  0.141139   
4  blues.00004.wav      30          0.308598         0.087850  0.091540   

    rms_var  spec_cent_mean  spec_cent_var  spec_bw_mean    spec_bw_var  ...  \
0  0.002830     1784.416546  129739.837401   2002.657106   85829.345908  ...   
1  0.002374     1529.871314  376011.650101   2038.612143  213889.873843  ...   
2  0.002753     1552.637786  156538.294709   1747.382028   76150.688778  ...   
3  0.006340     1070.110059  184506.655421   1596.244204  166637.568438  ...   
4  0.002305     1835.507009  343334.237041   1748.367477   88460.061228  ...   

   mfcc16_var  mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfc

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 3:</b>
    
Welche Spalten sind als Features für das neuonale Netz ungeeignet? Entfernen Sie diese Spalten aus dem Dataframe.
    
</div>

In [ ]:
# Identifizieren der nicht-numerischen Spalten im DataFrame
non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns

# Entfernen aller nicht-numerischen Spalten außer 'label'
columns_to_drop = [col for col in non_numeric_columns if col != 'label']
df.drop(columns=columns_to_drop, inplace=True)
print(df.columns)
df.drop(columns=['length'], inplace=True)

# So könnte ein für das Modell aussagekräftigeres Feature entstehen
print("Entfernte Spalten:", columns_to_drop)
print(df.dtypes)
print(df.head())

Index(['length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spec_cent_mean', 'spec_cent_var', 'spec_bw_mean', 'spec_bw_var',
       'rolloff_mean', 'rolloff_var', 'zcr_mean', 'zcr_var', 'harmony_mean',
       'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean',
       'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var',
       'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean',
       'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var',
       'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean',
       'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var',
       'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean',
       'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var',
       'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label'],
      dtype='object')
Entfernte Spalten: ['filename']
chroma_stft_mean    float

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 4:</b>
    
Wandeln Sie die Spalte label mit Hilfe der Bibliothek "sklearn.preprocessing" und des "LabelEncoder()" in Zahlenwerte um.
    
</div>

In [46]:
# Ausgabe der in df existierenden Spalten
print("Verfügbare Spalten:", df.columns)

# Falls die Spalte 'label' vorhanden ist, wird sie enkodiert
if 'label' in df.columns:
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label'])
    print(df.head())
else:
    print("Spalte 'label' nicht gefunden. Bitte überprüfe den Spaltennamen.")

Verfügbare Spalten: Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spec_cent_mean', 'spec_cent_var', 'spec_bw_mean', 'spec_bw_var',
       'rolloff_mean', 'rolloff_var', 'zcr_mean', 'zcr_var', 'harmony_mean',
       'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean',
       'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var',
       'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean',
       'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var',
       'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean',
       'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var',
       'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean',
       'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var',
       'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var', 'label'],
      dtype='object')
   chroma_stft_mean  chroma_stft_var  rms_mean 

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 5:</b>
    
Skalieren Sie alle passenden Werte mit Hilfe der Bibliothek "sklearn.preprocessing" und des "StandardScaler()".
    
</div>

In [47]:
# code
from sklearn.preprocessing import StandardScaler

# Initialisieren des StandardScalers
scaler = StandardScaler()

# Skalieren aller numerischen Spalten außer der Zielspalte 'label'
numeric_columns = df.select_dtypes(include=[np.number]).columns
columns_to_scale = [col for col in numeric_columns if col != 'label']

df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# Ausgabe der skalierten Werte
print(df.head())

   chroma_stft_mean  chroma_stft_var  rms_mean   rms_var  spec_cent_mean  \
0         -0.353326         0.310527 -0.009918 -0.060767       -0.584047   
1         -0.463630         1.113497 -0.532379 -0.186311       -0.939687   
2         -0.186606        -0.138363  0.680115 -0.082164       -0.907879   
3          0.319097         0.990764  0.156829  0.904732       -1.582047   
4         -0.859836         0.194041 -0.598658 -0.205287       -0.512666   

   spec_cent_var  spec_bw_mean  spec_bw_var  rolloff_mean  rolloff_var  ...  \
0      -0.848699     -0.456189    -0.530287     -0.486930    -0.661423  ...   
1      -0.234104     -0.387872     0.797626     -0.650451     0.795445  ...   
2      -0.781821     -0.941229    -0.630649     -0.973070    -0.744521  ...   
3      -0.712023     -1.228402     0.307647     -1.516824    -0.245263  ...   
4      -0.315653     -0.939357    -0.503008     -0.630114    -0.190554  ...   

   mfcc16_var  mfcc17_mean  mfcc17_var  mfcc18_mean  mfcc18_var  mfc

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 6:</b>
    
Teilen Sie das Dataset in Trainings- und Testdaten auf und beschränken Sie die Testdaten auf 10%.
    
</div>

In [48]:
# code
from sklearn.model_selection import train_test_split

# Annahme: 'label' ist die Zielspalte
X = df.drop(columns=['label'])  # Features
y = df['label']  # Zielvariable


# Aufteilen in Trainings- und Testdaten (10% Testdaten)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Ausgabe der Größen der Datensätze
print(f"Trainingsdaten: {X_train.shape}, Testdaten: {X_test.shape}")

Trainingsdaten: (899, 57), Testdaten: (100, 57)


<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 7:</b>
    
Erstellen Sie mit "keras.models.Sequential()" ein Model mit mehreren Layern. Zwischen den Layern soll ein Dropout von 30% und eine BatchNormalization() durchgeführt werden. 
    
</div>

In [49]:
# code
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

# Initialisieren des Modells
model = Sequential()

# Hinzufügen der Eingabeschicht (Anzahl der Eingaben = Anzahl der Features)
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

# Hinzufügen von BatchNormalization und Dropout
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Hinzufügen einer versteckten Schicht
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Hinzufügen einer weiteren versteckten Schicht
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Hinzufügen der Ausgabeschicht (Anzahl der Ausgaben = Anzahl der Klassen)
model.add(Dense(len(y.unique()), activation='softmax'))

# Zusammenfassung des Modells
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 128)               7424      
                                                                 
 batch_normalization_12 (Bat  (None, 128)              512       
 chNormalization)                                                
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 batch_normalization_13 (Bat  (None, 64)               256       
 chNormalization)                                                
                                                                 
 dropout_13 (Dropout)        (None, 64)               

<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 8:</b>
    
Konfigurieren Sie das Model so, damit folgende Parameter verwendet werden: optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']
</div>

In [50]:
# code
# Kompilieren des Modells mit den angegebenen Parametern
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Ausgabe der Modellkonfiguration
print("Modell erfolgreich konfiguriert.")

Modell erfolgreich konfiguriert.


<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 9:</b>
    
Trainieren Sie das Model mit 200 Epochen und einer batch_size von 32. Bei Bedarf kann auch ein EarlyStopping Callback eingesetzt werden. Für das Training wird die Methode fit() verwendet.
</div>

In [51]:
# code
from keras.callbacks import EarlyStopping

# Optional: EarlyStopping Callback, um das Training zu stoppen, wenn sich die Validierungsleistung nicht verbessert
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Trainieren des Modells
history = model.fit(X_train, y_train, 
                    epochs=200, 
                    batch_size=32, 
                    validation_split=0.1, 
                    callbacks=[early_stopping], 
                    verbose=1)

# Ausgabe der Trainingshistorie
print("Training abgeschlossen.")

Epoch 1/200
26/26 [==============================] - 2s 26ms/step - loss: 2.6558 - accuracy: 0.1842 - val_loss: 2.0916 - val_accuracy: 0.2667
Epoch 2/200
26/26 [==============================] - 0s 6ms/step - loss: 2.1695 - accuracy: 0.2917 - val_loss: 1.8670 - val_accuracy: 0.3778
Epoch 3/200
26/26 [==============================] - 0s 5ms/step - loss: 1.8301 - accuracy: 0.3733 - val_loss: 1.7125 - val_accuracy: 0.4333
Epoch 4/200
26/26 [==============================] - 0s 5ms/step - loss: 1.7145 - accuracy: 0.3993 - val_loss: 1.5585 - val_accuracy: 0.4556
Epoch 5/200
26/26 [==============================] - 0s 5ms/step - loss: 1.6123 - accuracy: 0.4277 - val_loss: 1.4249 - val_accuracy: 0.5111
Epoch 6/200
26/26 [==============================] - 0s 5ms/step - loss: 1.5057 - accuracy: 0.4821 - val_loss: 1.3243 - val_accuracy: 0.5333
Epoch 7/200
26/26 [==============================] - 0s 5ms/step - loss: 1.4338 - accuracy: 0.4981 - val_loss: 1.2292 - val_accuracy: 0.5111
Epoch 8/200


<div class="alert alert-block alert-success">
&#128187; <b>Aufgabe 10:</b>
    
Evaluieren Sie das Model mit der evaluate() Methode und geben Sie den 'loss' und die 'accuracy' aus.
</div>

In [52]:
# code
# Evaluieren des Modells auf den Testdaten
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

# Ausgabe von Loss und Accuracy
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

4/4 [==============================] - 0s 3ms/step - loss: 0.7310 - accuracy: 0.7700
Loss: 0.7310336828231812
Accuracy: 0.7699999809265137
